In [4]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = '../model/bottleneck_fc_model.h5'

#small
#train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
#validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
#nb_train_samples = 1462
#nb_validation_samples = 167
total_num_classes = 18

#verysmall
train_data_dir = '../data/verysmall/train'
validation_data_dir = '../data/verysmall/val'
nb_train_samples = 157
nb_validation_samples = 18
total_num_classes = 2

epochs = 50
batch_size = 16


In [5]:
datagen = ImageDataGenerator(rescale=1. / 255)

# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
bottleneck_features_train = model.predict_generator(
    generator, (nb_train_samples // batch_size)+1)
np.save(open('../model/bottleneck_features_train.npy', 'w'),
        bottleneck_features_train)

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
bottleneck_features_validation = model.predict_generator(
    generator, (nb_validation_samples // batch_size)+1)
np.save(open('../model/bottleneck_features_validation.npy', 'w'),
        bottleneck_features_validation)

Found 157 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


In [6]:
from keras.utils.np_utils import to_categorical

#Get data
train_data = np.load(open('../model/bottleneck_features_train.npy'))
validation_data = np.load(open('../model/bottleneck_features_validation.npy'))

#TRAIN DATA
#full
# train_labels = np.array([0]*684+[1]*598+[2]*655+[3]*657+[4]*808+[5]*675+[6]*715+[7]*946+[8]*995+[9]*1021+[10]*803+[11]*816+[12]*566+[13]*959+[14]*842+[15]*831+[16]*849+[17]*893)
#small
#train_labels = np.array([0]*78+[1]*79+[2]*77+[3]*85+[4]*78+[5]*87+[6]*80+[7]*81+[8]*81+[9]*80+[10]*85+[11]*83+[12]*74+[13]*87+[14]*83+[15]*83+[16]*78+[17]*83)
#verysmall
train_labels = np.array([0]*78+[1]*79)

#VALIDATION DATA
#full
#validation_labels = np.array([0]*72+[1]*73+[2]*72+[3]*93+[4]*78+[5]*74+[6]*85+[7]*124+[8]*131+[9]*118+[10]*109+[11]*105+[12]*80+[13]*130+[14]*108+[15]*89+[16]*111+[17]*120)
#small
#validation_labels = np.array([0]*8+[1]*10+[2]*11+[3]*5+[4]*11+[5]*6+[6]*8+[7]*8+[8]*9+[9]*12+[10]*7+[11]*10+[12]*14+[13]*5+[14]*11+[15]*11+[16]*12+[17]*9)
#verysmall
validation_labels = np.array([0]*8+[1]*10)

# to_categorical
train_labels = to_categorical(train_labels, num_classes=total_num_classes)
validation_labels = to_categorical(validation_labels, num_classes=total_num_classes)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(total_num_classes, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path,overwrite=True)

Train on 157 samples, validate on 18 samples
Epoch 1/50
157/157 [==============================] - 0s - loss: 7.3043 - acc: 0.5032 - val_loss: 7.1636 - val_acc: 0.5556
Epoch 2/50
157/157 [==============================] - 0s - loss: 8.0077 - acc: 0.5032 - val_loss: 7.1636 - val_acc: 0.5556
Epoch 3/50
157/157 [==============================] - 0s - loss: 8.0077 - acc: 0.5032 - val_loss: 7.1636 - val_acc: 0.5556
Epoch 4/50
157/157 [==============================] - 0s - loss: 8.0077 - acc: 0.5032 - val_loss: 7.1636 - val_acc: 0.5556
Epoch 5/50
157/157 [==============================] - 0s - loss: 8.0077 - acc: 0.5032 - val_loss: 7.1636 - val_acc: 0.5556
Epoch 6/50
157/157 [==============================] - 0s - loss: 8.0077 - acc: 0.5032 - val_loss: 7.1636 - val_acc: 0.5556
Epoch 7/50
157/157 [==============================] - 0s - loss: 7.9830 - acc: 0.5032 - val_loss: 5.0586 - val_acc: 0.5556
Epoch 8/50
157/157 [==============================] - 0s - loss: 8.8912 - acc: 0.4331 - val_lo